In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Make sure plots display inline in Jupyter
%matplotlib inline

# Configure pandas and Jupyter to show full output (no truncation)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Configure Jupyter to show full output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Increase output limit (if using Jupyter)
import sys
try:
    sys.stdout.reconfigure(encoding='utf-8')
except (AttributeError, TypeError):
    # sys.stdout.reconfigure() not available in Jupyter notebooks
    pass

# ───────────────────────────────────────────────
# MEV Analysis #2: Top 10 Toxic MEV Stats & Validator Breakdown (Expanded)
# ───────────────────────────────────────────────
# Uses df_clean (cleaned fused table after time parsing & deletion)
# Assumes df_clean already exists in memory from previous steps
# ───────────────────────────────────────────────

df_clean = pd.read_parquet('/Users/aileen/Downloads/pamm/pamm_clean_final.parquet')
print(f"Using cleaned fused table for MEV Analysis #2. Total rows: {len(df_clean):,}")
print("Available columns:", df_clean.columns.tolist())

# ───────────────────────────────────────────────
# Parse trades column to extract from_token and to_token
# ───────────────────────────────────────────────
import json
import ast

def parse_trades(trades_item):
    """
    Safely parse trades column to extract from_token and to_token.
    Returns: (from_token, to_token)
    """
    # Handle None and NaN values safely
    if trades_item is None:
        return None, None
    try:
        if pd.isna(trades_item):
            return None, None
    except (ValueError, TypeError):
        # If pd.isna returns array, check if all are NA
        if isinstance(trades_item, (list, np.ndarray)):
            return None, None
        pass
    
    try:
        # Handle string representation of list/dict
        if isinstance(trades_item, str):
            # Try to parse as Python literal (handles single quotes)
            try:
                parsed = ast.literal_eval(trades_item)
            except (ValueError, SyntaxError):
                # Fallback to JSON parsing
                cleaned = trades_item.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
                parsed = json.loads(cleaned)
        else:
            parsed = trades_item
        
        # Handle list format
        if isinstance(parsed, list) and len(parsed) > 0:
            trade_dict = parsed[0] if isinstance(parsed[0], dict) else parsed
            if isinstance(trade_dict, dict):
                from_token = trade_dict.get('from_token')
                to_token = trade_dict.get('to_token')
                return from_token, to_token
        
        # Handle dict format
        elif isinstance(parsed, dict):
            from_token = parsed.get('from_token')
            to_token = parsed.get('to_token')
            return from_token, to_token
            
    except (json.JSONDecodeError, ValueError, AttributeError, TypeError) as e:
        pass
    
    return None, None

# Apply parsing function
print("\nParsing trades column to extract from_token and to_token...")
parsed_trades_series = df_clean['trades'].apply(parse_trades)
parsed_trades_df = pd.DataFrame(
    parsed_trades_series.tolist(),
    columns=['from_token', 'to_token'],
    index=df_clean.index
)

# Add parsed columns to dataframe
df_clean['from_token'] = parsed_trades_df['from_token']
df_clean['to_token'] = parsed_trades_df['to_token']

# ───────────────────────────────────────────────
# Map token addresses to human-readable names
# ───────────────────────────────────────────────
# Common Solana token address to name mapping
TOKEN_NAME_MAP = {
    # Wrapped SOL
    'So11111111111111111111111111111111111111112': 'WSOL',
    'So11111111111111111111111111111111111111111': 'WSOL',  # Alternative WSOL address
    
    # Stablecoins
    'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v': 'USDC',
    'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': 'USDT',
    '7kbnvuGBxxj8AG9qp8Scn56muWGaRaFqxg1FsRp3PaFT': 'UXD',
    'EchesyfXePKdLbiA9xYJ6UXTx4urmzE2pz6DXuTVg3fH': 'UXP',
    
    # Popular tokens
    '9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump': 'PUMP',  # Based on example in data
    'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So': 'mSOL',
    '7vfCXTUXx5WJV5JADk17DUJ4ksgau7utNKj4b963voxs': 'ETH',
    '2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk': 'ETH',
    '7dHbWXmci3dT8UFYWYZweBLXgycu7Y3iL6trKn1Y7ARj': 'ORCA',
    '4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R': 'RAY',
    'SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt': 'SRM',
    'DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263': 'BONK',
    'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm': 'WIF',
    'JUPyiwrY2skib1qAwsWzJxZxqJp3ZJ5K3JqJ5JqJ5JqJ5': 'JUP',
    
    # Add more common tokens as needed
    # You can extend this mapping based on tokens found in your dataset
}

def get_token_name(token_address):
    """
    Map token address to human-readable name.
    Returns token name if found, otherwise returns address (truncated if long).
    """
    if pd.isna(token_address) or token_address is None:
        return None
    
    token_address_str = str(token_address)
    
    # Check direct mapping
    if token_address_str in TOKEN_NAME_MAP:
        return TOKEN_NAME_MAP[token_address_str]
    
    # Return address if not found (can be extended to query on-chain metadata)
    return token_address_str

# Apply token name mapping
print("\nMapping token addresses to names...")
df_clean['from_token_name'] = df_clean['from_token'].apply(get_token_name)
df_clean['to_token_name'] = df_clean['to_token'].apply(get_token_name)

# Show token name mapping statistics
trades_with_names = df_clean[df_clean['kind'] == 'TRADE']
from_token_mapped = trades_with_names['from_token_name'].notna().sum()
to_token_mapped = trades_with_names['to_token_name'].notna().sum()

# Count how many are mapped vs unmapped (addresses)
from_token_named = trades_with_names[
    (trades_with_names['from_token_name'].notna()) & 
    (~trades_with_names['from_token_name'].str.match(r'^[A-Za-z0-9]{32,44}$', na=False))
].shape[0]

to_token_named = trades_with_names[
    (trades_with_names['to_token_name'].notna()) & 
    (~trades_with_names['to_token_name'].str.match(r'^[A-Za-z0-9]{32,44}$', na=False))
].shape[0]

print(f"Token name mapping statistics:")
print(f"  from_token_name: {from_token_named:,} tokens mapped to names")
print(f"  to_token_name: {to_token_named:,} tokens mapped to names")

# Show most common token pairs
if from_token_mapped > 0 and to_token_mapped > 0:
    token_pairs = trades_with_names[
        trades_with_names['from_token_name'].notna() & 
        trades_with_names['to_token_name'].notna()
    ][['from_token_name', 'to_token_name']].copy()
    
    if len(token_pairs) > 0:
        token_pairs['pair'] = token_pairs['from_token_name'] + '/' + token_pairs['to_token_name']
        top_pairs = token_pairs['pair'].value_counts().head(10)
        print(f"\nTop 10 token pairs (by name):")
        for pair, count in top_pairs.items():
            print(f"  {pair}: {count:,} trades")

# Calculate parsing success rate
trades_with_tokens = df_clean[df_clean['kind'] == 'TRADE']
success_rate = trades_with_tokens['from_token'].notna().mean() * 100
print(f"\nParsing success rate for TRADE events (from_token non-null): {success_rate:.2f}%")
print(f"Total TRADE events with token pairs: {trades_with_tokens['from_token'].notna().sum():,}")

# ───────────────────────────────────────────────
# Expanded Detection: Add Classic Sandwich (3 TRADEs) + Fat (≥5)
# ───────────────────────────────────────────────
# Ensure lag columns for back-running
df_clean['prev_kind'] = df_clean['kind'].shift(1)
df_clean['prev_ms_time'] = df_clean['ms_time'].shift(1)
df_clean['time_diff_ms'] = df_clean['ms_time'] - df_clean['prev_ms_time']

# Extract TRADE records
trades = df_clean[df_clean['kind'] == 'TRADE'].copy()
print(f"\nTRADE events for MEV analysis: {len(trades):,}")

# 1. IMPROVED Sandwich Detection with False Positive Filtering
# Distinguishes MEV attacks from:
# - Jupiter aggregator routing (multiple users, same aggregator)
# - Legitimate market makers
# - Failed sandwich attempts (front-run but no profitable back-run)

sandwich_records = []
failed_attempts = []  # Track failed sandwich attempts

for slot, g in trades.groupby('slot'):
    trade_count = len(g)
    if trade_count < 3:
        continue
    
    # Sort by time within slot
    g = g.sort_values('ms_time').reset_index(drop=True)
    
    signer_cnt = g['signer'].value_counts()
    attackers = signer_cnt[signer_cnt >= 2].index  # Signers with ≥2 tx in slot
    
    if len(attackers) == 0:
        continue
    
    # Use amm_trade instead of amm_oracle
    amm_col = 'amm_trade' if 'amm_trade' in g.columns else 'amm_oracle'
    most_common_amm = g[amm_col].mode()
    amm_name = most_common_amm.iloc[0] if not most_common_amm.empty else 'Unknown'
    validator = g['validator'].mode().iloc[0] if 'validator' in g.columns and not g['validator'].mode().empty else 'Unknown'
    
    # Check for aggregator patterns (Jupiter, etc.)
    # Aggregators typically have:
    # - Many unique signers (users) in same slot
    # - Similar transaction timing
    # - Same aggregator program ID (if available)
    unique_signers = g['signer'].nunique()
    aggregator_ratio = unique_signers / trade_count
    
    for attacker in attackers:
        attacker_trades = g[g['signer'] == attacker].copy()
        attacker_tx_count = len(attacker_trades)
        
        # Filter out aggregator routing:
        # If >70% of trades have unique signers, likely aggregator routing
        if aggregator_ratio > 0.7 and attacker_tx_count < trade_count * 0.3:
            # This looks like aggregator routing, not MEV
            continue
        
        # Check for sandwich pattern: front-run → victim(s) → back-run
        # MEV sandwich requires:
        # 1. Attacker has trades at beginning AND end of slot
        # 2. Other signers (victims) in between
        attacker_positions = attacker_trades.index.tolist()
        first_attacker_pos = min(attacker_positions)
        last_attacker_pos = max(attacker_positions)
        
        # Check if there are other signers between first and last attacker trade
        middle_trades = g.iloc[first_attacker_pos+1:last_attacker_pos]
        victims_between = middle_trades[middle_trades['signer'] != attacker]
        
        if len(victims_between) == 0:
            # No victims between attacker trades - could be failed attempt or unrelated
            # Check if this is a failed attempt (attacker at start but no back-run)
            if first_attacker_pos < len(g) * 0.3:  # Attacker in first 30% of slot
                failed_attempts.append({
                    'slot': slot,
                    'amm_trade': amm_name,
                    'attacker_signer': attacker,
                    'type': 'failed_frontrun',
                    'validator': validator,
                    'reason': 'no_victims_between'
                })
            continue
        
        # NEW: Validate token pair consistency for sandwich detection
        # In a true sandwich, all trades should be on the same token pair
        # This helps distinguish MEV from aggregator routing across different pools
        token_pairs_valid = True
        if 'from_token' in g.columns and 'to_token' in g.columns:
            # Get token pairs for all trades in slot
            attacker_token_pairs = attacker_trades[['from_token', 'to_token']].dropna()
            victim_token_pairs = victims_between[['from_token', 'to_token']].dropna()
            
            if len(attacker_token_pairs) > 0 and len(victim_token_pairs) > 0:
                # Check if attacker trades use consistent token pairs
                attacker_pairs = attacker_token_pairs.apply(
                    lambda x: tuple(sorted([str(x['from_token']), str(x['to_token'])])), axis=1
                ).unique()
                
                # Check if victims use same token pairs as attacker
                victim_pairs = victim_token_pairs.apply(
                    lambda x: tuple(sorted([str(x['from_token']), str(x['to_token'])])), axis=1
                ).unique()
                
                # For a valid sandwich, at least one token pair should be shared
                # (attacker front-runs and back-runs on same pair, victims trade on same pair)
                common_pairs = set(attacker_pairs) & set(victim_pairs)
                
                if len(common_pairs) == 0 and len(attacker_pairs) > 0:
                    # No common token pairs - might be aggregator routing across pools
                    # Still allow if attacker has consistent pair (could be reverse sandwich)
                    if len(attacker_pairs) > 1:
                        token_pairs_valid = False  # Attacker trading multiple pairs
                # If common_pairs exist or attacker has single consistent pair, it's valid
        
        # Additional validation: Check timing patterns
        # MEV attacks typically have:
        # - Front-run early in slot
        # - Back-run late in slot
        # - Victims in between
        attacker_times = attacker_trades['ms_time'].values
        slot_start = g['ms_time'].min()
        slot_end = g['ms_time'].max()
        slot_duration = slot_end - slot_start
        
        # Check if attacker trades span the slot (indicates sandwich)
        time_span = attacker_times.max() - attacker_times.min()
        span_ratio = time_span / slot_duration if slot_duration > 0 else 0
        
        # MEV sandwich should span at least 30% of slot duration
        if span_ratio < 0.3 and trade_count < 5:
            # Too tight timing, might be aggregator bundle
            continue
        
        # Additional validation: Token pair consistency
        # If token pairs are available but don't match, this might not be a true sandwich
        if not token_pairs_valid:
            # Skip if token pairs indicate this is not a sandwich on same pool
            continue
        
        # Check for consistent pattern across multiple slots (indicates bot, not aggregator)
        # This will be checked later in aggregation
        
        # Extract token pair info for the record
        token_pair_info = None
        if 'from_token' in attacker_trades.columns and 'to_token' in attacker_trades.columns:
            attacker_tokens = attacker_trades[['from_token', 'to_token']].dropna()
            if len(attacker_tokens) > 0:
                # Get most common token pair
                token_pairs = attacker_tokens.apply(
                    lambda x: f"{x['from_token']}/{x['to_token']}", axis=1
                )
                token_pair_info = token_pairs.mode().iloc[0] if len(token_pairs.mode()) > 0 else None
        
        # Classify as sandwich
        sandwich_type = 'fat_sandwich' if trade_count >= 5 else 'sandwich'
        
        sandwich_records.append({
            'slot': slot,
            'amm_trade': amm_name,
            'attacker_signer': attacker,
            'attack_tx_count': attacker_tx_count,
            'trade_count_in_slot': trade_count,
            'victims_count': len(victims_between),
            'unique_signers': unique_signers,
            'aggregator_ratio': aggregator_ratio,
            'time_span_ratio': span_ratio,
            'token_pair': token_pair_info,  # NEW: Add token pair info
            'type': sandwich_type,
            'validator': validator,
            'confidence': 'high' if span_ratio > 0.5 and len(victims_between) >= 2 else 'medium'
        })

sandwich_df = pd.DataFrame(sandwich_records)
failed_attempts_df = pd.DataFrame(failed_attempts) if failed_attempts else pd.DataFrame()

print(f"\nSandwich patterns detected: {len(sandwich_df)}")
print(f"Failed sandwich attempts (front-run only): {len(failed_attempts_df)}")
if not sandwich_df.empty:
    print(f"High confidence MEV attacks: {len(sandwich_df[sandwich_df['confidence'] == 'high'])}")
    print(f"Medium confidence (possible aggregator): {len(sandwich_df[sandwich_df['confidence'] == 'medium'])}")

# Get top 10 attackers - prioritize high-confidence MEV attacks
if not sandwich_df.empty:
    # Filter to high-confidence attacks first, then medium
    high_conf = sandwich_df[sandwich_df['confidence'] == 'high']
    
    if len(high_conf) > 0:
        # Count by high-confidence attacks
        high_conf_counts = high_conf['attacker_signer'].value_counts()
        print(f"\nHigh-confidence MEV attackers: {len(high_conf_counts)}")
        
        # Get top 10 from high-confidence, fill with medium if needed
        top10_high = high_conf_counts.head(10).index.tolist()
        
        if len(top10_high) < 10:
            # Add medium-confidence attackers to reach 10
            medium_conf = sandwich_df[sandwich_df['confidence'] == 'medium']
            medium_counts = medium_conf['attacker_signer'].value_counts()
            # Exclude already selected
            medium_counts = medium_counts[~medium_counts.index.isin(top10_high)]
            top10_medium = medium_counts.head(10 - len(top10_high)).index.tolist()
            top10_attackers = top10_high + top10_medium
        else:
            top10_attackers = top10_high
    else:
        # Fallback to all if no high-confidence
        top10_attackers = sandwich_df['attacker_signer'].value_counts().head(10).index.tolist()
        print("\n⚠️  WARNING: No high-confidence MEV attacks found. Top 10 may include aggregator routing.")
    
    print(f"\nTop 10 attackers (prioritizing high-confidence MEV): {top10_attackers}")
    
    # Show breakdown
    if not sandwich_df.empty:
        print(f"\nConfidence breakdown:")
        print(f"  High confidence (likely MEV): {len(sandwich_df[sandwich_df['confidence'] == 'high'])}")
        print(f"  Medium confidence (possible aggregator/MEV): {len(sandwich_df[sandwich_df['confidence'] == 'medium'])}")
else:
    top10_attackers = []
    print("No sandwich patterns detected.")

# ───────────────────────────────────────────────
# VERIFICATION: Classify all detected attackers to filter out aggregators/wash traders
# This runs BEFORE final top 10 selection to ensure we only show actual MEV bots
# ───────────────────────────────────────────────
print(f"\n{'='*80}")
print("VERIFICATION: Classifying All Detected Attackers")
print(f"{'='*80}")
print("⚠️  Running verification on all detected attackers to filter out:")
print("   - Aggregators (Jupiter routing)")
print("   - Wash traders (volume inflation)")
print("   - Regular trade bots")
print("   Only addresses classified as 'LIKELY MEV BOT' or 'POSSIBLE MEV' will be included in final top 10.\n")

# Get all unique attackers from sandwich detection
all_detected_attackers = []
if not sandwich_df.empty:
    all_detected_attackers = sandwich_df['attacker_signer'].unique().tolist()

verification_results = []

if len(all_detected_attackers) > 0:
    print(f"Verifying {len(all_detected_attackers)} detected attackers...\n")
    
    for idx, attacker in enumerate(all_detected_attackers, 1):
        if idx % 50 == 0:
            print(f"  Progress: {idx}/{len(all_detected_attackers)} attackers verified...")
        
        attacker_trades = trades[trades['signer'] == attacker].copy()
        
        if len(attacker_trades) == 0:
            continue
        
        # Calculate metrics
        total_trades = len(attacker_trades)
        
        # 1. Check for aggregator pattern: many unique signers in same slots
        attacker_slots = attacker_trades['slot'].unique()
        aggregator_score = 0
        
        # Sample up to 200 slots for aggregator detection
        sample_slots = min(200, len(attacker_slots))
        for slot in attacker_slots[:sample_slots]:
            slot_trades = trades[trades['slot'] == slot]
            unique_signers = slot_trades['signer'].nunique()
            total_in_slot = len(slot_trades)
            if total_in_slot > 0:
                ratio = unique_signers / total_in_slot
                if ratio > 0.7:  # High unique signer ratio = aggregator
                    aggregator_score += 1
        
        aggregator_likelihood = aggregator_score / sample_slots if sample_slots > 0 else 0
        
        # 2. Check for MEV patterns
        # - Late-slot timing (front-running)
        late_slot_trades = attacker_trades[attacker_trades['us_since_first_shred'] > 300000]
        late_slot_ratio = len(late_slot_trades) / total_trades if total_trades > 0 else 0
        
        # - Oracle back-running
        attacker_trades_with_prev = attacker_trades.copy()
        attacker_trades_with_prev['prev_kind'] = df_clean.loc[attacker_trades_with_prev.index, 'prev_kind'].values
        oracle_backrun = attacker_trades_with_prev[
            (attacker_trades_with_prev['prev_kind'] == 'ORACLE') & 
            (attacker_trades_with_prev['time_diff_ms'] < 50)
        ]
        oracle_backrun_ratio = len(oracle_backrun) / total_trades if total_trades > 0 else 0
        
        # - High bytes_changed (oracle manipulation)
        high_bytes = attacker_trades[attacker_trades['bytes_changed_trade'] > 50]
        high_bytes_ratio = len(high_bytes) / total_trades if total_trades > 0 else 0
        
        # - Clusters (multiple tx in same slot)
        attacker_trades['tx_in_slot'] = attacker_trades.groupby('slot')['slot'].transform('count')
        clusters = attacker_trades[attacker_trades['tx_in_slot'] >= 2]
        cluster_ratio = len(clusters) / total_trades if total_trades > 0 else 0
        
        # Calculate MEV score
        mev_score = (late_slot_ratio * 0.3 + 
                    oracle_backrun_ratio * 0.3 + 
                    high_bytes_ratio * 0.2 + 
                    cluster_ratio * 0.2)
        
        # 3. Check for wash trading patterns
        if len(attacker_trades) > 10:
            time_span_hours = (attacker_trades['ms_time'].max() - attacker_trades['ms_time'].min()) / (1000 * 60 * 60)
            trades_per_hour = len(attacker_trades) / time_span_hours if time_span_hours > 0 else 0
            wash_trading_score = trades_per_hour / 50  # Normalize to 50 trades/hour
        else:
            wash_trading_score = 0
            trades_per_hour = 0
        
        # Classification with wash trading detection
        if aggregator_likelihood > 0.5:
            classification = "LIKELY AGGREGATOR (Jupiter, etc.)"
            confidence = "High" if aggregator_likelihood > 0.7 else "Medium"
        elif wash_trading_score > 1.0 and mev_score < 0.2:
            classification = "LIKELY WASH TRADING (Volume Inflation)"
            confidence = "High" if wash_trading_score > 2.0 else "Medium"
        elif mev_score > 0.3:
            classification = "LIKELY MEV BOT"
            confidence = "High" if mev_score > 0.5 else "Medium"
        elif cluster_ratio > 0.3:
            classification = "POSSIBLE MEV (Sandwich patterns)"
            confidence = "Medium"
        else:
            classification = "REGULAR TRADE BOT / UNKNOWN"
            confidence = "Low"
        
        verification_results.append({
            'attacker': attacker,
            'total_trades': total_trades,
            'trades_per_hour': trades_per_hour,
            'aggregator_likelihood': aggregator_likelihood,
            'late_slot_ratio': late_slot_ratio,
            'oracle_backrun_ratio': oracle_backrun_ratio,
            'high_bytes_ratio': high_bytes_ratio,
            'cluster_ratio': cluster_ratio,
            'mev_score': mev_score,
            'wash_trading_score': wash_trading_score,
            'classification': classification,
            'confidence': confidence
        })
    
    # Create verification dataframe
    verification_df = pd.DataFrame(verification_results)
    
    if not verification_df.empty:
        # Save full verification results
        verification_df.to_csv('mev_attacker_verification.csv', index=False)
        print(f"\n✓ Verification complete. Results saved to: mev_attacker_verification.csv")
        
        # Show classification breakdown
        print(f"\nClassification Breakdown (All {len(verification_df)} Detected Attackers):")
        classification_counts = verification_df['classification'].value_counts()
        for classification, count in classification_counts.items():
            print(f"  {classification}: {count}")
        
        # Filter to only MEV bots for top 10 list
        mev_bots = verification_df[
            verification_df['classification'].isin(['LIKELY MEV BOT', 'POSSIBLE MEV (Sandwich patterns)'])
        ].copy()
        
        if len(mev_bots) > 0:
            # Sort by MEV score (descending) and get top 10
            mev_bots = mev_bots.sort_values('mev_score', ascending=False)
            top10_mev_bots = mev_bots.head(10)['attacker'].tolist()
            
            print(f"\n{'='*80}")
            print("FILTERED TOP 10 MEV ATTACKERS (After Verification)")
            print(f"{'='*80}")
            print(f"\n✓ Found {len(mev_bots)} addresses classified as MEV bots (out of {len(verification_df)} total)")
            print(f"✓ Top 10 MEV bots (by MEV score):")
            for idx, attacker in enumerate(top10_mev_bots, 1):
                attacker_info = mev_bots[mev_bots['attacker'] == attacker].iloc[0]
                print(f"  {idx}. {attacker}")
                print(f"     Classification: {attacker_info['classification']}")
                print(f"     MEV Score: {attacker_info['mev_score']:.2%}")
                print(f"     Total Trades: {attacker_info['total_trades']:,}")
            
            # Update top10_attackers to only include verified MEV bots
            top10_attackers = top10_mev_bots
            
            print(f"\n⚠️  IMPORTANT: Updated top10_attackers list to only include verified MEV bots.")
            print(f"   Excluded: {len(verification_df) - len(mev_bots)} addresses classified as aggregators/wash traders/regular bots")
        else:
            print(f"\n⚠️  WARNING: No addresses classified as 'LIKELY MEV BOT' or 'POSSIBLE MEV' found!")
            print(f"   All {len(verification_df)} detected addresses are classified as:")
            for classification, count in classification_counts.items():
                print(f"     - {classification}: {count}")
            print(f"\n   This suggests:")
            print(f"     - Most activity is aggregator routing (Jupiter, etc.)")
            print(f"     - Or detection thresholds may need adjustment")
            print(f"   Original top10_attackers list (by sandwich count) will be used, but may include aggregators.")
    else:
        print("No verification results generated.")
else:
    print("No attackers detected for verification.")

# Analyze failed attempts
if not failed_attempts_df.empty:
    print(f"\n{'='*80}")
    print("FAILED SANDWICH ATTEMPTS ANALYSIS")
    print(f"{'='*80}")
    print(f"Total failed attempts: {len(failed_attempts_df)}")
    print(f"Failed attempts per attacker:")
    print(failed_attempts_df['attacker_signer'].value_counts().head(10))
    print("\n⚠️  Note: Failed attempts indicate front-running without profitable back-run.")
    print("   These may be:")
    print("   - Unsuccessful MEV attempts")
    print("   - Legitimate early trades that didn't result in sandwich")
    print("   - Aggregator routing with timing coincidences")

# 2. Front-running Detection (late-slot trades >0.3s delay, expanded to include slot avg latency)
front_running = trades[trades['us_since_first_shred'] > 300000].copy()
front_running['type'] = 'front_running'
# Use amm_trade instead of amm_oracle
front_running = front_running[['slot', 'amm_trade', 'signer', 'type', 'validator', 'us_since_first_shred']].rename(columns={'signer': 'attacker_signer'})
if 'amm_trade' not in front_running.columns:
    front_running = front_running.rename(columns={'amm_oracle': 'amm_trade'})
front_running_top10 = front_running[front_running['attacker_signer'].isin(top10_attackers)]

# Average latency per slot in front-running
front_running['slot_avg_latency'] = front_running.groupby('slot')['us_since_first_shred'].transform('mean')

# 3. Back-running Detection (immediately after ORACLE, <50ms, expanded to include oracle count)
back_running = trades[(trades['prev_kind'] == 'ORACLE') & (trades['time_diff_ms'] < 50)].copy()
back_running['type'] = 'back_running'
# Use amm_trade instead of amm_oracle
back_running = back_running[['slot', 'amm_trade', 'signer', 'type', 'validator', 'time_diff_ms']].rename(columns={'signer': 'attacker_signer'})
if 'amm_trade' not in back_running.columns:
    back_running = back_running.rename(columns={'amm_oracle': 'amm_trade'})
back_running_top10 = back_running[back_running['attacker_signer'].isin(top10_attackers)]

# Count oracle updates per slot in back-running
oracle_per_slot = df_clean[df_clean['kind'] == 'ORACLE'].groupby('slot').size().reset_index(name='oracle_count')
back_running = back_running.merge(oracle_per_slot, on='slot', how='left').fillna(0)

# Merge all MEV records (top10 only)
# Ensure all dataframes have amm_trade column before merging
if 'amm_trade' not in sandwich_df.columns and 'amm' in sandwich_df.columns:
    sandwich_df = sandwich_df.rename(columns={'amm': 'amm_trade'})

all_mev = pd.concat([sandwich_df, front_running_top10, back_running_top10], ignore_index=True)

# Function to display Per pAMM Top 10 MEV table
def display_per_pamm_top10_mev(stats_df, top_n=10):
    """
    Display Top N MEV attackers per pAMM in a formatted table.
    
    Parameters:
    -----------
    stats_df : DataFrame
        Stats dataframe with columns: amm_trade, attacker_signer, validator, 
        fat_sandwich, sandwich, front_running, back_running, etc.
    top_n : int
        Number of top attackers to show per AMM (default: 10)
    """
    from IPython.display import display as ipy_display
    
    if stats_df.empty:
        print("\nNo MEV stats to display.")
        return
    
    # Ensure amm_trade column exists
    if 'amm_trade' not in stats_df.columns:
        if 'amm_oracle' in stats_df.columns:
            stats_df = stats_df.rename(columns={'amm_oracle': 'amm_trade'})
        else:
            print("Error: No amm_trade or amm_oracle column found in stats.")
            return
    
    print(f"\n{'='*100}")
    print(f"Per pAMM Top {top_n} MEV Attackers Table")
    print(f"{'='*100}\n")
    
    # Group by amm_trade
    amm_groups = stats_df.groupby('amm_trade')
    
    for amm, group in amm_groups:
        # Sort by total MEV activity (fat_sandwich + sandwich + front_running + back_running)
        group = group.copy()  # Avoid SettingWithCopyWarning
        group['total_mev'] = (group.get('fat_sandwich', 0) + 
                             group.get('sandwich', 0) + 
                             group.get('front_running', 0) + 
                             group.get('back_running', 0))
        
        # Get top N per AMM
        top_group = group.nlargest(top_n, 'total_mev')
        
        print(f"\n{'─'*100}")
        print(f"pAMM: {amm}")
        print(f"Total Attackers: {len(group)}, Showing Top {min(top_n, len(group))}")
        print(f"{'─'*100}")
        
        # Display columns
        display_cols = ['attacker_signer', 'validator', 'fat_sandwich', 'sandwich', 
                       'front_running', 'back_running', 'sandwich_complete', 
                       'cost_sol', 'profit_sol', 'net_profit_sol']
        available_cols = [c for c in display_cols if c in top_group.columns]
        
        # Format the table
        display_df = top_group[available_cols].copy()
        
        # Format float columns
        float_cols = ['cost_sol', 'profit_sol', 'net_profit_sol']
        for col in float_cols:
            if col in display_df.columns:
                display_df[col] = display_df[col].apply(lambda x: f'{x:.4f}' if isinstance(x, (int, float)) else x)
        
        # Reset index for display
        display_df = display_df.reset_index(drop=True)
        display_df.index = display_df.index + 1  # Start from 1 instead of 0
        
        # Use display() for better Jupyter formatting (scrollable, no truncation)
        ipy_display(display_df)
        print()

if all_mev.empty:
    print("\nNo MEV patterns detected.")
    stats = pd.DataFrame()
else:
    # Ensure all_mev has amm_trade column (should already have it from previous fixes)
    if 'amm_trade' not in all_mev.columns:
        if 'amm_oracle' in all_mev.columns:
            all_mev = all_mev.rename(columns={'amm_oracle': 'amm_trade'})
        elif 'amm' in all_mev.columns:
            all_mev = all_mev.rename(columns={'amm': 'amm_trade'})
    
    # Stats: Group by amm_trade + attacker_signer + type (FIXED: use amm_trade not amm_oracle)
    # IMPORTANT: Don't filter to top10 attackers here - we want ALL pAMMs with their top attackers
    stats = all_mev.groupby(['amm_trade', 'attacker_signer', 'type']).size().unstack(fill_value=0)
    stats = stats.reset_index()

    # Safe fill missing type columns
    for col in ['fat_sandwich', 'sandwich', 'front_running', 'back_running']:
        if col not in stats.columns:
            stats[col] = 0

    # Add validator (first per attacker-amm pair)
    validator_map = all_mev.groupby(['amm_trade', 'attacker_signer'])['validator'].first().reset_index(name='validator')
    stats = pd.merge(stats, validator_map, on=['amm_trade', 'attacker_signer'], how='left')

    # Derived metrics (approximate SOL)
    stats['sandwich_complete'] = stats.get('fat_sandwich', 0) // 2 + stats.get('sandwich', 0) // 2
    stats['cost_sol'] = (stats['sandwich_complete'] * 0.001) + (stats.get('front_running', 0) * 0.0005) + (stats.get('back_running', 0) * 0.0005)
    stats['profit_sol'] = (stats['sandwich_complete'] * 0.01) + (stats.get('front_running', 0) * 0.002) + (stats.get('back_running', 0) * 0.002)
    stats['net_profit_sol'] = stats['profit_sol'] - stats['cost_sol']
    
    # Add confidence levels to stats if available
    if 'confidence' in sandwich_df.columns:
        confidence_map = sandwich_df.groupby(['amm_trade', 'attacker_signer'])['confidence'].agg(
            lambda x: 'high' if (x == 'high').any() else 'medium'
        ).reset_index(name='confidence')
        stats = pd.merge(stats, confidence_map, on=['amm_trade', 'attacker_signer'], how='left')
    
    # Save ALL stats (not filtered to top10)
    stats.to_csv('per_pamm_all_mev_with_validator.csv', index=False)
    print("\nAll pAMM MEV stats (all attackers) saved to: per_pamm_all_mev_with_validator.csv")
    
    # Save failed attempts
    if not failed_attempts_df.empty:
        failed_attempts_df.to_csv('failed_sandwich_attempts.csv', index=False)
        print("Failed sandwich attempts saved to: failed_sandwich_attempts.csv")
    
    # Save confidence breakdown
    if not sandwich_df.empty and 'confidence' in sandwich_df.columns:
        confidence_summary = sandwich_df.groupby(['amm_trade', 'confidence']).size().unstack(fill_value=0)
        confidence_summary.to_csv('mev_confidence_breakdown.csv')
        print("MEV confidence breakdown saved to: mev_confidence_breakdown.csv")
        print("\n⚠️  IMPORTANT: Review confidence levels:")
        print("   - 'high': Likely MEV attacks (attacker spans slot, victims between)")
        print("   - 'medium': Possible aggregator routing or less clear MEV pattern")

# ───────────────────────────────────────────────
# Variable Ranking by MEV Detection Contribution
# ───────────────────────────────────────────────
print(f"\n{'='*80}")
print("VARIABLE RANKING BY MEV DETECTION CONTRIBUTION")
print(f"{'='*80}")

# Calculate contribution scores for each variable
variable_contributions = {}

if not sandwich_df.empty:
    # 1. Signer (attacker identification) - CRITICAL
    # Used in: All MEV detection (sandwich, front-run, back-run)
    signer_contribution = {
        'variable': 'signer',
        'importance': 'CRITICAL',
        'score': 10.0,
        'usage': 'Identifies MEV attackers across all detection methods',
        'details': [
            'Sandwich: Groups trades by signer to find attackers with ≥2 tx in slot',
            'Front-run: Filters to top MEV attackers by signer',
            'Back-run: Filters to top MEV attackers by signer',
            'Fat sandwich: Validates same signer at start and end of slot'
        ]
    }
    variable_contributions['signer'] = signer_contribution
    
    # 2. Token Pair (from_token/to_token) - CRITICAL for validation
    # Used in: Sandwich validation to ensure same pool
    token_pair_usage = 0
    if 'token_pair' in sandwich_df.columns:
        token_pair_usage = sandwich_df['token_pair'].notna().sum()
    
    token_pair_contribution = {
        'variable': 'from_token/to_token',
        'importance': 'CRITICAL',
        'score': 9.5,
        'usage': 'Validates sandwich attacks are on same token pair (same pool)',
        'details': [
            'Prevents false positives from aggregator routing across different pools',
            'Ensures front-run and back-run are on same trading pair',
            'Helps distinguish true MEV from multi-pool aggregator bundles',
            f'Used in {token_pair_usage:,} sandwich detections'
        ]
    }
    variable_contributions['token_pair'] = token_pair_contribution
    
    # 3. Slot - CRITICAL
    # Used in: All detection methods (grouping trades by slot)
    slot_contribution = {
        'variable': 'slot',
        'importance': 'CRITICAL',
        'score': 10.0,
        'usage': 'Groups trades temporally for sandwich detection',
        'details': [
            'Sandwich: Groups trades in same slot to detect front-run → victim → back-run',
            'Front-run: Identifies late-slot trades (>300ms)',
            'Back-run: Links trades to oracle updates in same slot',
            'Fat sandwich: Requires ≥5 trades in same slot'
        ]
    }
    variable_contributions['slot'] = slot_contribution
    
    # 4. Time (ms_time) - CRITICAL
    # Used in: Timing validation for sandwich and back-run
    time_contribution = {
        'variable': 'ms_time',
        'importance': 'CRITICAL',
        'score': 9.0,
        'usage': 'Validates timing patterns for MEV attacks',
        'details': [
            'Sandwich: Validates attacker trades span ≥30% of slot duration',
            'Back-run: Requires <50ms response time after oracle update',
            'Front-run: Identifies late-slot trades (>300ms delay)',
            'Time span ratio: Key metric for high-confidence MEV (span >50% = high confidence)'
        ]
    }
    variable_contributions['ms_time'] = time_contribution
    
    # 5. Validator - HIGH
    # Used in: Attribution and clustering
    validator_contribution = {
        'variable': 'validator',
        'importance': 'HIGH',
        'score': 7.5,
        'usage': 'Attribution and validator-level MEV analysis',
        'details': [
            'Identifies which validators process MEV attacks',
            'Enables validator-level clustering and analysis',
            'Helps identify validator-MEV bot relationships',
            'Used in per-validator MEV statistics'
        ]
    }
    variable_contributions['validator'] = validator_contribution
    
    # 6. AMM (amm_trade) - HIGH
    # Used in: Protocol-level analysis
    amm_contribution = {
        'variable': 'amm_trade',
        'importance': 'HIGH',
        'score': 7.0,
        'usage': 'Identifies which pAMM protocols are targeted',
        'details': [
            'Groups MEV attacks by protocol (ZeroFi, HumidiFi, etc.)',
            'Enables per-protocol MEV statistics',
            'Helps identify protocol-specific attack patterns',
            'Used in per-pAMM top 10 attacker analysis'
        ]
    }
    variable_contributions['amm_trade'] = amm_contribution
    
    # 7. Time difference (time_diff_ms) - HIGH
    # Used in: Back-running detection
    time_diff_contribution = {
        'variable': 'time_diff_ms',
        'importance': 'HIGH',
        'score': 8.0,
        'usage': 'Detects back-running after oracle updates',
        'details': [
            'Back-run: Requires <50ms between oracle update and trade',
            'Indicates automated bot response to oracle price changes',
            'Key metric for distinguishing MEV from normal trading',
            'Threshold: <50ms = suspicious, <30ms = highly suspicious'
        ]
    }
    variable_contributions['time_diff_ms'] = time_diff_contribution
    
    # 8. Latency (us_since_first_shred) - MEDIUM-HIGH
    # Used in: Front-running detection
    latency_contribution = {
        'variable': 'us_since_first_shred',
        'importance': 'MEDIUM-HIGH',
        'score': 7.0,
        'usage': 'Detects late-slot trades (front-running)',
        'details': [
            'Front-run: Requires >300ms delay (300,000 microseconds)',
            'Indicates strategic placement late in slot',
            'Suggests mempool monitoring and insertion',
            'Threshold: >300ms = suspicious front-running'
        ]
    }
    variable_contributions['us_since_first_shred'] = latency_contribution
    
    # 9. Previous kind (prev_kind) - MEDIUM
    # Used in: Back-running detection
    prev_kind_contribution = {
        'variable': 'prev_kind',
        'importance': 'MEDIUM',
        'score': 6.5,
        'usage': 'Links trades to preceding oracle updates',
        'details': [
            'Back-run: Requires prev_kind == "ORACLE"',
            'Validates trade immediately follows price update',
            'Key for detecting oracle exploitation',
            'Used in conjunction with time_diff_ms'
        ]
    }
    variable_contributions['prev_kind'] = prev_kind_contribution
    
    # 10. Aggregator ratio - MEDIUM
    # Used in: Filtering false positives
    aggregator_ratio_contribution = {
        'variable': 'aggregator_ratio',
        'importance': 'MEDIUM',
        'score': 6.0,
        'usage': 'Filters out Jupiter aggregator routing (false positives)',
        'details': [
            'Calculated as: unique_signers / trade_count',
            'If >0.7 and attacker has <30% of trades → likely aggregator',
            'Prevents false positives from legitimate aggregator routing',
            'Key for distinguishing MEV from aggregator bundles'
        ]
    }
    variable_contributions['aggregator_ratio'] = aggregator_ratio_contribution
    
    # 11. Time span ratio - MEDIUM
    # Used in: Confidence scoring
    time_span_ratio_contribution = {
        'variable': 'time_span_ratio',
        'importance': 'MEDIUM',
        'score': 6.5,
        'usage': 'Confidence scoring for sandwich attacks',
        'details': [
            'Calculated as: (attacker_max_time - attacker_min_time) / slot_duration',
            'High confidence: >50% span AND ≥2 victims',
            'Medium confidence: 30-50% span OR 1 victim',
            'Validates attacker trades span significant portion of slot'
        ]
    }
    variable_contributions['time_span_ratio'] = time_span_ratio_contribution
    
    # 12. Victims count - MEDIUM
    # Used in: Confidence scoring
    victims_count_contribution = {
        'variable': 'victims_count',
        'importance': 'MEDIUM',
        'score': 6.0,
        'usage': 'Confidence scoring and fat sandwich validation',
        'details': [
            'High confidence: ≥2 victims between attacker trades',
            'Validates true sandwich pattern (victims between front-run and back-run)',
            'Fat sandwich: Multiple victims in same slot',
            'Key for distinguishing MEV from failed attempts'
        ]
    }
    variable_contributions['victims_count'] = victims_count_contribution

# Sort by score (descending)
ranked_variables = sorted(variable_contributions.items(), key=lambda x: x[1]['score'], reverse=True)

# Display ranking
print("\nRank | Variable              | Importance | Score | Usage")
print("-" * 80)
for rank, (var_name, var_info) in enumerate(ranked_variables, 1):
    print(f"{rank:4d} | {var_info['variable']:20s} | {var_info['importance']:11s} | {var_info['score']:5.1f} | {var_info['usage']}")

# Save ranking to file
ranking_df = pd.DataFrame([
    {
        'rank': rank,
        'variable': var_info['variable'],
        'importance': var_info['importance'],
        'score': var_info['score'],
        'usage': var_info['usage'],
        'details': '; '.join(var_info['details'])
    }
    for rank, (var_name, var_info) in enumerate(ranked_variables, 1)
])
ranking_df.to_csv('mev_variable_ranking.csv', index=False)
print(f"\n✓ Variable ranking saved to: mev_variable_ranking.csv")

# Display detailed breakdown
print(f"\n{'='*80}")
print("DETAILED VARIABLE CONTRIBUTION BREAKDOWN")
print(f"{'='*80}")
for rank, (var_name, var_info) in enumerate(ranked_variables, 1):
    print(f"\n{rank}. {var_info['variable']} (Score: {var_info['score']:.1f}, {var_info['importance']})")
    print(f"   Usage: {var_info['usage']}")
    print("   Details:")
    for detail in var_info['details']:
        print(f"     • {detail}")

# Stats and all_mev are now available for next cells


Using cleaned fused table for MEV Analysis #2. Total rows: 5,506,090
Available columns: ['slot', 'time', 'validator', 'tx_idx', 'sig', 'signer', 'kind', 'amm_oracle', 'account_updates', 'trades', 'us_since_first_shred', 'amm_trade', 'account_trade', 'is_pool_trade', 'bytes_changed_trade', 'datetime', 'timing_missing', 'ms_time']

Parsing trades column to extract from_token and to_token...

Mapping token addresses to names...
Token name mapping statistics:
  from_token_name: 0 tokens mapped to names
  to_token_name: 0 tokens mapped to names

Parsing success rate for TRADE events (from_token non-null): 0.00%
Total TRADE events with token pairs: 0

TRADE events for MEV analysis: 683,828

Sandwich patterns detected: 26223
Failed sandwich attempts (front-run only): 6339
High confidence MEV attacks: 7897
Medium confidence (possible aggregator): 18326

High-confidence MEV attackers: 197

Top 10 attackers (prioritizing high-confidence MEV): ['YubQzu18FDqJRyNfG8JqHmsdbxhnoQqcKUHBdUkN6tP', 'E2MP

In [ ]:
# ───────────────────────────────────────────────
# Cell 2: Display Per pAMM Top 10 MEV Attackers Table
# ───────────────────────────────────────────────
if not stats.empty:
    # Show summary of pAMMs found
    unique_pamms = stats['amm_trade'].unique()
    print(f"\nTotal pAMMs with MEV activity: {len(unique_pamms)}")
    print(f"pAMMs found: {', '.join(sorted(unique_pamms))}\n")
    
    # Display Per pAMM Top 10 MEV table using the function
    # Use display() instead of print() for better formatting in Jupyter
    display_per_pamm_top10_mev(stats, top_n=10)
    
    # Also save filtered top10 per pAMM version
    # For each pAMM, get top 10 attackers
    top10_per_pamm_list = []
    for amm, group in stats.groupby('amm_trade'):
        group = group.copy()  # Avoid SettingWithCopyWarning
        group['total_mev'] = (group.get('fat_sandwich', 0) + 
                             group.get('sandwich', 0) + 
                             group.get('front_running', 0) + 
                             group.get('back_running', 0))
        top10_per_pamm = group.nlargest(10, 'total_mev')
        top10_per_pamm_list.append(top10_per_pamm)
    
    if top10_per_pamm_list:
        top10_per_pamm_df = pd.concat(top10_per_pamm_list, ignore_index=True)
        top10_per_pamm_df = top10_per_pamm_df.drop('total_mev', axis=1, errors='ignore')
        top10_per_pamm_df.to_csv('per_pamm_top10_mev_with_validator.csv', index=False)
        print("\n✓ Per pAMM Top 10 MEV stats (top 10 per pAMM) saved to: per_pamm_top10_mev_with_validator.csv")
        
        # Also display the full dataframe using display() for better viewing
        print("\n" + "="*100)
        print("Full Per pAMM Top 10 MEV Table (All pAMMs Combined)")
        print("="*100)
        display(top10_per_pamm_df)
else:
    print("\nNo MEV stats available to display.")

In [ ]:
# ───────────────────────────────────────────────
# Cell 3: Trades and Bots Group by Validator
# ───────────────────────────────────────────────
print("\n" + "="*100)
print("Trades and Bots Group by Validator")
print("="*100 + "\n")

if not all_mev.empty:
    trades_per_validator = trades.groupby('validator').size().reset_index(name='trade_count')
    
    bots_per_validator = all_mev.groupby('validator')['attacker_signer'].nunique().reset_index(name='bot_count')
    
    mev_types_per_validator = all_mev.groupby('validator')['type'].value_counts().unstack(fill_value=0)
    
    validator_stats = pd.merge(trades_per_validator, bots_per_validator, on='validator', how='outer').fillna(0)
    validator_stats = pd.merge(validator_stats, mev_types_per_validator, on='validator', how='left').fillna(0)
    
    validator_stats['bot_ratio_%'] = (validator_stats['bot_count'] / validator_stats['trade_count'] * 100).round(2)
    
    validator_stats = validator_stats.sort_values('bot_count', ascending=False)
    
    # Safe output columns
    output_cols = ['validator', 'trade_count', 'bot_count', 'bot_ratio_%']
    mev_types = ['fat_sandwich', 'sandwich', 'front_running', 'back_running']
    for col in mev_types:
        if col in validator_stats.columns:
            output_cols.append(col)
    
    # Show Top 10 Validators
    print("Top 10 Validators by Bot Count:")
    print("-" * 100)
    top10_validator_stats = validator_stats.head(10)
    print(top10_validator_stats[output_cols].to_string(index=False, float_format=lambda x: f'{x:.2f}' if isinstance(x, float) else x))
    print()
    
    # Also display using display() for better viewing in Jupyter
    print("\n" + "="*100)
    print("Top 10 Validators Table (Formatted)")
    print("="*100)
    display(top10_validator_stats[output_cols])
    
    # Save full table
    validator_stats.to_csv('mev_trades_bots_per_validator.csv', index=False)
    print(f"\n✓ Full Trades and Bots grouped by validator saved to: mev_trades_bots_per_validator.csv")
    print(f"✓ Total validators: {len(validator_stats)}")
    
    # Also show full table using display() if user wants to see all
    print("\n" + "="*100)
    print("Full Validator Stats Table (All Validators)")
    print("="*100)
    display(validator_stats[output_cols])
else:
    print("No MEV data available for validator analysis.")

In [ ]:
# ───────────────────────────────────────────────
# Cell 4: VERIFICATION - Are These Actually MEV Bots or Aggregators?
# Based on Grok's analysis: Verify if detected "attackers" are MEV bots or just aggregators
# ───────────────────────────────────────────────

print("\n" + "="*100)
print("VERIFICATION: MEV Bots vs Aggregators (Jupiter, etc.)")
print("="*100)
print("\n⚠️  IMPORTANT: The detected 'attackers' may be:")
print("   1. MEV bots (actual attackers)")
print("   2. Jupiter aggregator (bundling user orders)")
print("   3. Regular trade bots (market making, arbitrage)")
print("   4. Legitimate high-frequency traders")
print("\nThis analysis helps distinguish between them.\n")

if not sandwich_df.empty and len(top10_attackers) > 0:
    # Analyze top attackers in detail
    print("Analyzing top detected 'attackers' for MEV vs Aggregator patterns...\n")
    
    verification_results = []
    
    for attacker in top10_attackers[:5]:  # Analyze top 5
        attacker_trades = trades[trades['signer'] == attacker].copy()
        
        if len(attacker_trades) == 0:
            continue
        
        # Calculate metrics
        total_trades = len(attacker_trades)
        
        # 1. Check for aggregator pattern: many unique signers in same slots
        attacker_slots = attacker_trades['slot'].unique()
        aggregator_score = 0
        
        for slot in attacker_slots[:100]:  # Sample first 100 slots
            slot_trades = trades[trades['slot'] == slot]
            unique_signers = slot_trades['signer'].nunique()
            total_in_slot = len(slot_trades)
            if total_in_slot > 0:
                ratio = unique_signers / total_in_slot
                if ratio > 0.7:  # High unique signer ratio = aggregator
                    aggregator_score += 1
        
        aggregator_likelihood = aggregator_score / min(100, len(attacker_slots))
        
        # 2. Check for MEV patterns
        # - Late-slot timing (front-running)
        late_slot_trades = attacker_trades[attacker_trades['us_since_first_shred'] > 300000]
        late_slot_ratio = len(late_slot_trades) / total_trades if total_trades > 0 else 0
        
        # - Oracle back-running
        attacker_trades_with_prev = attacker_trades.copy()
        attacker_trades_with_prev['prev_kind'] = df_clean.loc[attacker_trades_with_prev.index - 1, 'kind'].values
        oracle_backrun = attacker_trades_with_prev[
            (attacker_trades_with_prev['prev_kind'] == 'ORACLE') & 
            (attacker_trades_with_prev['time_diff_ms'] < 50)
        ]
        oracle_backrun_ratio = len(oracle_backrun) / total_trades if total_trades > 0 else 0
        
        # - High bytes_changed (oracle manipulation)
        high_bytes = attacker_trades[attacker_trades['bytes_changed_trade'] > 50]
        high_bytes_ratio = len(high_bytes) / total_trades if total_trades > 0 else 0
        
        # - Clusters (multiple tx in same slot)
        attacker_trades['tx_in_slot'] = attacker_trades.groupby('slot')['slot'].transform('count')
        clusters = attacker_trades[attacker_trades['tx_in_slot'] >= 2]
        cluster_ratio = len(clusters) / total_trades if total_trades > 0 else 0
        
        # Calculate MEV score
        mev_score = (late_slot_ratio * 0.3 + 
                    oracle_backrun_ratio * 0.3 + 
                    high_bytes_ratio * 0.2 + 
                    cluster_ratio * 0.2)
        
        # 3. Check for wash trading patterns
        # Wash trading indicators:
        # - Circular flows (same address sending/receiving)
        # - Many unique recipient addresses
        # - Low net profit despite high volume
        # - Temporary addresses (created and used once)
        # - Immediate return of funds
        
        # Check for circular flow pattern: same signer appearing multiple times in short period
        attacker_trades_sorted = attacker_trades.sort_values('ms_time')
        time_gaps = attacker_trades_sorted['ms_time'].diff()
        
        # Wash trading: rapid cycling (many trades in short time)
        if len(attacker_trades) > 10:
            time_span_hours = (attacker_trades['ms_time'].max() - attacker_trades['ms_time'].min()) / (1000 * 60 * 60)
            trades_per_hour = len(attacker_trades) / time_span_hours if time_span_hours > 0 else 0
            
            # High trade frequency with low profit indicators = wash trading
            # Orb analysis showed: 116K USDC cycled, only 3.33 USDC profit = wash trading
            # Pattern: >50 trades/hour with low MEV score = likely wash trading
            wash_trading_score = trades_per_hour / 50  # Normalize to 50 trades/hour
        else:
            wash_trading_score = 0
            trades_per_hour = 0
        
        # Classification with wash trading detection
        if aggregator_likelihood > 0.5:
            classification = "LIKELY AGGREGATOR (Jupiter, etc.)"
            confidence = "High" if aggregator_likelihood > 0.7 else "Medium"
        elif wash_trading_score > 1.0 and mev_score < 0.2:
            # High trade frequency but low MEV score = wash trading
            classification = "LIKELY WASH TRADING (Volume Inflation)"
            confidence = "High" if wash_trading_score > 2.0 else "Medium"
        elif mev_score > 0.3:
            classification = "LIKELY MEV BOT"
            confidence = "High" if mev_score > 0.5 else "Medium"
        elif cluster_ratio > 0.3:
            classification = "POSSIBLE MEV (Sandwich patterns)"
            confidence = "Medium"
        else:
            classification = "REGULAR TRADE BOT / UNKNOWN"
            confidence = "Low"
        
        verification_results.append({
            'attacker': attacker,
            'total_trades': total_trades,
            'trades_per_hour': f"{trades_per_hour:.1f}",
            'aggregator_likelihood': f"{aggregator_likelihood:.2%}",
            'late_slot_ratio': f"{late_slot_ratio:.2%}",
            'oracle_backrun_ratio': f"{oracle_backrun_ratio:.2%}",
            'high_bytes_ratio': f"{high_bytes_ratio:.2%}",
            'cluster_ratio': f"{cluster_ratio:.2%}",
            'mev_score': f"{mev_score:.2%}",
            'wash_trading_score': f"{wash_trading_score:.2f}",
            'classification': classification,
            'confidence': confidence
        })
        
        print(f"\n{'─'*100}")
        print(f"Attacker: {attacker}")
        print(f"{'─'*100}")
        print(f"Total Trades: {total_trades:,}")
        print(f"\nPattern Analysis:")
        print(f"  Trades per Hour: {trades_per_hour:.1f}")
        print(f"  Aggregator Likelihood: {aggregator_likelihood:.2%} (slots with >70% unique signers)")
        print(f"  Late-Slot Trades (>300ms): {late_slot_ratio:.2%}")
        print(f"  Oracle Back-Running (<50ms): {oracle_backrun_ratio:.2%}")
        print(f"  High Bytes Changed (>50): {high_bytes_ratio:.2%}")
        print(f"  Clusters (≥2 tx/slot): {cluster_ratio:.2%}")
        print(f"\nMEV Score: {mev_score:.2%}")
        print(f"Wash Trading Score: {wash_trading_score:.2f} (>1.0 = suspicious)")
        print(f"\nClassification: {classification}")
        print(f"Confidence: {confidence}")
        
        # Special note for wash trading
        if "WASH TRADING" in classification:
            print(f"\n⚠️  WASH TRADING DETECTED:")
            print(f"   - High trade frequency ({trades_per_hour:.1f} trades/hour)")
            print(f"   - Low MEV score ({mev_score:.2%}) = not extracting value")
            print(f"   - Pattern: Volume inflation, transaction count farming, or protocol gaming")
            print(f"   - Example: AEB9dXBoxkrapNd59Kg29JefMMf3M1WLcNA12XjKSf4R cycled 116K USDC, only 3.33 USDC profit")
    
    # Summary table
    verification_df = pd.DataFrame(verification_results)
    if not verification_df.empty:
        print(f"\n{'='*100}")
        print("VERIFICATION SUMMARY")
        print(f"{'='*100}")
        display(verification_df)
        
        # Save verification results
        verification_df.to_csv('mev_attacker_verification.csv', index=False)
        print("\n✓ Verification results saved to: mev_attacker_verification.csv")
        
        # Count classifications
        print(f"\nClassification Breakdown:")
        print(verification_df['classification'].value_counts())
        
        print(f"\n⚠️  IMPORTANT NOTES:")
        print(f"   - If 'LIKELY AGGREGATOR': These are NOT MEV bots, just Jupiter/aggregator routing")
        print(f"   - If 'LIKELY WASH TRADING': Volume inflation, not MEV (e.g., AEB9dXBoxkrapNd59Kg29JefMMf3M1WLcNA12XjKSf4R)")
        print(f"   - If 'LIKELY MEV BOT': These show MEV attack patterns")
        print(f"   - If 'REGULAR TRADE BOT': May be legitimate market making/arbitrage")
        print(f"   - Check Orb (orbmarkets.io) for these addresses to verify transaction history")
        print(f"   - Look for failed transactions (slippage errors) = failed MEV attempts")
        print(f"   - Wash trading: Circular flows, low profit despite high volume = protocol gaming")
        
else:
    print("No attackers detected to verify. This may indicate:")
    print("  - Low MEV activity (good news for pAMMs)")
    print("  - Detection thresholds too strict")
    print("  - Mostly aggregator routing (Jupiter) without MEV patterns")

# ───────────────────────────────────────────────
# Detailed Activity Analysis for Top Attackers
# ───────────────────────────────────────────────
if len(top10_attackers) > 0:
    print(f"\n{'='*100}")
    print("DETAILED ACTIVITY ANALYSIS")
    print(f"{'='*100}\n")
    
    for attacker in top10_attackers[:3]:  # Top 3
        attacker_txs = df_clean[df_clean['signer'] == attacker].copy()
        
        if len(attacker_txs) == 0:
            continue
        
        print(f"\nAttacker: {attacker}")
        print(f"Total Events: {len(attacker_txs):,}")
        
        # Event type breakdown
        event_types = attacker_txs['kind'].value_counts()
        print(f"\nEvent Types:")
        for kind, count in event_types.items():
            print(f"  {kind}: {count:,}")
        
        # TRADE-specific analysis
        attacker_trades = attacker_txs[attacker_txs['kind'] == 'TRADE']
        if len(attacker_trades) > 0:
            print(f"\nTRADE Analysis:")
            print(f"  Total TRADEs: {len(attacker_trades):,}")
            
            # Timing analysis
            if 'us_since_first_shred' in attacker_trades.columns:
                avg_latency = attacker_trades['us_since_first_shred'].mean()
                late_trades = len(attacker_trades[attacker_trades['us_since_first_shred'] > 300000])
                print(f"  Avg Latency: {avg_latency:,.0f} µs")
                print(f"  Late-Slot Trades (>300ms): {late_trades:,} ({late_trades/len(attacker_trades):.2%})")
            
            # Bytes changed analysis
            if 'bytes_changed_trade' in attacker_trades.columns:
                high_bytes = len(attacker_trades[attacker_trades['bytes_changed_trade'] > 50])
                print(f"  High Bytes Changed (>50): {high_bytes:,} ({high_bytes/len(attacker_trades):.2%})")
            
            # Slot clustering
            slot_counts = attacker_trades['slot'].value_counts()
            multi_slot = len(slot_counts[slot_counts >= 2])
            print(f"  Slots with Multiple Trades: {multi_slot:,}")
            
            # Save detailed activity
            activity_file = f'attacker_{attacker[:8]}_detailed_activity.csv'
            attacker_trades.to_csv(activity_file, index=False)
            print(f"  ✓ Detailed activity saved to: {activity_file}")
    
    # ───────────────────────────────────────────────
    # Pool Activity Analysis - Show Bot Density Across Pools
    # ───────────────────────────────────────────────
    print(f"\n{'='*100}")
    print("POOL ACTIVITY ANALYSIS - Attacker Distribution Across Pools")
    print(f"{'='*100}\n")
    
    if len(top10_attackers) > 0:
        pool_analysis_results = []
        
        for attacker in top10_attackers:
            attacker_trades = trades[trades['signer'] == attacker].copy()
            
            if len(attacker_trades) == 0:
                continue
            
            # Get pool information from account_trade (where is_pool_trade == True)
            if 'account_trade' in attacker_trades.columns and 'is_pool_trade' in attacker_trades.columns:
                pool_trades = attacker_trades[attacker_trades['is_pool_trade'] == True]
                
                if len(pool_trades) > 0:
                    # Get unique pools
                    unique_pools = pool_trades['account_trade'].nunique()
                    pool_counts = pool_trades['account_trade'].value_counts()
                    
                    # Top pools for this attacker
                    top_pools = pool_counts.head(5).to_dict()
                    
                    pool_analysis_results.append({
                        'attacker': attacker,
                        'total_trades': len(attacker_trades),
                        'pool_trades': len(pool_trades),
                        'unique_pools': unique_pools,
                        'pools_per_trade_ratio': unique_pools / len(pool_trades) if len(pool_trades) > 0 else 0,
                        'top_pool': pool_counts.index[0] if len(pool_counts) > 0 else 'N/A',
                        'top_pool_trades': pool_counts.iloc[0] if len(pool_counts) > 0 else 0,
                        'top_pool_%': (pool_counts.iloc[0] / len(pool_trades) * 100) if len(pool_trades) > 0 else 0
                    })
                else:
                    # No pool trades detected
                    pool_analysis_results.append({
                        'attacker': attacker,
                        'total_trades': len(attacker_trades),
                        'pool_trades': 0,
                        'unique_pools': 0,
                        'pools_per_trade_ratio': 0,
                        'top_pool': 'N/A',
                        'top_pool_trades': 0,
                        'top_pool_%': 0
                    })
            else:
                # Fallback: use amm_trade as proxy for pools
                unique_amms = attacker_trades['amm_trade'].nunique()
                amm_counts = attacker_trades['amm_trade'].value_counts()
                
                pool_analysis_results.append({
                    'attacker': attacker,
                    'total_trades': len(attacker_trades),
                    'pool_trades': len(attacker_trades),
                    'unique_pools': unique_amms,  # Using AMM as proxy
                    'pools_per_trade_ratio': unique_amms / len(attacker_trades) if len(attacker_trades) > 0 else 0,
                    'top_pool': amm_counts.index[0] if len(amm_counts) > 0 else 'N/A',
                    'top_pool_trades': amm_counts.iloc[0] if len(amm_counts) > 0 else 0,
                    'top_pool_%': (amm_counts.iloc[0] / len(attacker_trades) * 100) if len(attacker_trades) > 0 else 0
                })
        
        if pool_analysis_results:
            pool_analysis_df = pd.DataFrame(pool_analysis_results)
            
            print("Attacker Pool Distribution Summary:")
            print("-" * 100)
            display_cols = ['attacker', 'total_trades', 'unique_pools', 'top_pool', 'top_pool_trades', 'top_pool_%']
            display(pool_analysis_df[display_cols])
            
            # Detailed pool breakdown per attacker
            print(f"\n{'─'*100}")
            print("DETAILED POOL BREAKDOWN PER ATTACKER:")
            print(f"{'─'*100}\n")
            
            for attacker in top10_attackers:
                attacker_trades = trades[trades['signer'] == attacker].copy()
                
                if len(attacker_trades) == 0:
                    continue
                
                print(f"\nAttacker: {attacker}")
                print(f"{'─'*80}")
                
                if 'account_trade' in attacker_trades.columns and 'is_pool_trade' in attacker_trades.columns:
                    pool_trades = attacker_trades[attacker_trades['is_pool_trade'] == True]
                    
                    if len(pool_trades) > 0:
                        pool_stats = pool_trades.groupby(['account_trade', 'amm_trade']).agg({
                            'slot': 'count',
                            'ms_time': ['min', 'max']
                        }).reset_index()
                        pool_stats.columns = ['pool', 'amm', 'trade_count', 'first_trade', 'last_trade']
                        pool_stats = pool_stats.sort_values('trade_count', ascending=False)
                        
                        print(f"Active in {len(pool_stats)} pools:")
                        print(f"\nTop 10 Pools by Trade Count:")
                        top_pools = pool_stats.head(10)
                        for idx, row in top_pools.iterrows():
                            print(f"  {idx+1}. Pool: {row['pool'][:20]}... | AMM: {row['amm']} | Trades: {row['trade_count']:,}")
                        
                        if len(pool_stats) > 10:
                            print(f"  ... and {len(pool_stats) - 10} more pools")
                    else:
                        print("No pool trades detected (using AMM as proxy)")
                        amm_stats = attacker_trades.groupby('amm_trade').size().sort_values(ascending=False)
                        print(f"Active in {len(amm_stats)} AMMs:")
                        for amm, count in amm_stats.head(10).items():
                            print(f"  • {amm}: {count:,} trades")
                else:
                    # Fallback to AMM
                    amm_stats = attacker_trades.groupby('amm_trade').size().sort_values(ascending=False)
                    print(f"Active in {len(amm_stats)} AMMs (pool data not available):")
                    for amm, count in amm_stats.head(10).items():
                        print(f"  • {amm}: {count:,} trades")
            
            # Summary statistics
            print(f"\n{'─'*100}")
            print("POOL ACTIVITY SUMMARY:")
            print(f"{'─'*100}")
            print(f"Average pools per attacker: {pool_analysis_df['unique_pools'].mean():.1f}")
            print(f"Max pools for single attacker: {pool_analysis_df['unique_pools'].max()}")
            print(f"Min pools for single attacker: {pool_analysis_df['unique_pools'].min()}")
            print(f"\nAttacker Pool Concentration:")
            print(f"  High concentration (1-2 pools): {len(pool_analysis_df[pool_analysis_df['unique_pools'] <= 2])} attackers")
            print(f"  Medium concentration (3-5 pools): {len(pool_analysis_df[(pool_analysis_df['unique_pools'] > 2) & (pool_analysis_df['unique_pools'] <= 5)])} attackers")
            print(f"  Low concentration (6+ pools): {len(pool_analysis_df[pool_analysis_df['unique_pools'] > 5])} attackers")
            
            # Pool density analysis - which pools have most attackers
            print(f"\n{'─'*100}")
            print("POOL DENSITY - Which Pools Have Most Attackers:")
            print(f"{'─'*100}")
            
            # Collect all pool-attacker pairs
            pool_attacker_pairs = []
            for attacker in top10_attackers:
                attacker_trades = trades[trades['signer'] == attacker].copy()
                
                if 'account_trade' in attacker_trades.columns and 'is_pool_trade' in attacker_trades.columns:
                    pool_trades = attacker_trades[attacker_trades['is_pool_trade'] == True]
                    if len(pool_trades) > 0:
                        unique_pools = pool_trades['account_trade'].unique()
                        for pool in unique_pools:
                            pool_attacker_pairs.append({
                                'pool': pool,
                                'attacker': attacker,
                                'amm': pool_trades[pool_trades['account_trade'] == pool]['amm_trade'].iloc[0] if len(pool_trades[pool_trades['account_trade'] == pool]) > 0 else 'Unknown'
                            })
                else:
                    # Fallback to AMM
                    unique_amms = attacker_trades['amm_trade'].unique()
                    for amm in unique_amms:
                        pool_attacker_pairs.append({
                            'pool': amm,  # Using AMM as pool proxy
                            'attacker': attacker,
                            'amm': amm
                        })
            
            if pool_attacker_pairs:
                pool_density_df = pd.DataFrame(pool_attacker_pairs)
                pool_density = pool_density_df.groupby(['pool', 'amm']).size().reset_index(name='attacker_count')
                pool_density = pool_density.sort_values('attacker_count', ascending=False)
                
                print(f"\nTop 20 Pools by Attacker Count (Bot Density):")
                print(pool_density.head(20).to_string(index=False))
                
                # Save pool analysis
                pool_analysis_df.to_csv('attacker_pool_analysis.csv', index=False)
                pool_density.to_csv('pool_density_analysis.csv', index=False)
                print(f"\n✓ Attacker pool analysis saved to: attacker_pool_analysis.csv")
                print(f"✓ Pool density analysis saved to: pool_density_analysis.csv")
                
                # Insights
                print(f"\n{'─'*100}")
                print("INSIGHTS:")
                print(f"{'─'*100}")
                max_density_pool = pool_density.iloc[0]
                print(f"  • Most targeted pool: {max_density_pool['pool'][:20]}... ({max_density_pool['attacker_count']} attackers)")
                print(f"  • AMM: {max_density_pool['amm']}")
                print(f"  • High density pools (>3 attackers): {len(pool_density[pool_density['attacker_count'] > 3])} pools")
                print(f"  • Low density pools (1 attacker): {len(pool_density[pool_density['attacker_count'] == 1])} pools")
                print(f"\n  Interpretation:")
                print(f"    - High density pools (>3 attackers) = likely profitable targets for MEV")
                print(f"    - Low density (1 attacker) = specialized bot or less profitable")
                print(f"    - Multi-pool attackers = generalized bots scanning multiple opportunities")
    
    print(f"\n{'='*100}")
    print("RECOMMENDATIONS:")
    print(f"{'='*100}")
    print("1. Check Orb (orbmarkets.io) for these addresses:")
    for attacker in top10_attackers[:5]:
        print(f"   https://orbmarkets.io/account/{attacker}")
    print("\n2. Look for:")
    print("   - Failed transactions (slippage errors) = failed MEV attempts")
    print("   - Transaction patterns (repeated front-run/back-run)")
    print("   - Program IDs (Jupiter aggregator vs MEV bot)")
    print("   - Profit/loss patterns")
    print("\n3. Compare with known MEV bot addresses from public sources")
    print("4. Review verification results above to determine if aggregator or MEV")
    print("5. Review pool density analysis - high density pools may need protection")

In [ ]:
# Summary of saved files:
# 1. per_pamm_all_mev_with_validator.csv - All MEV stats for all attackers across all pAMMs
# 2. per_pamm_top10_mev_with_validator.csv - Top 10 attackers per pAMM
# 3. mev_trades_bots_per_validator.csv - Validator statistics with bot counts and MEV types